# Modelagem - Definição dos Modelos de Machine Learning
## Tech Challenge IADT - Fase 1

### Grupo
- Hiago Marques Rubio - RM
- Mylena Ferreira Lacerda - RM

---

## Objetivo

Este notebook é responsável pela **preparação dos dados** e **definição dos modelos**:

1. Carregamento e limpeza dos dados pré-processados
2. Remoção de features problemáticas (data leakage, multicolinearidade)
3. Definição e configuração de 3 modelos:
   - **Logistic Regression** (Baseline interpretável)
   - **Random Forest** (Robusto e confiável)
   - **XGBoost** (Estado da arte)

### Problema
- **Tipo:** Classificação Binária Supervisionada
- **Variável Alvo:** `diagnosed_diabetes` (0 = Sem diabetes, 1 = Com diabetes)
- **Dataset:** ~100.000 exemplos pré-processados

---

**Próximo passo:** Execute o notebook `treinamento.ipynb` para treinar e avaliar os modelos.

## 1. Importação de Bibliotecas

In [1]:
# Manipulação de dados
import pandas as pd
import numpy as np

# Visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

# Métricas
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    confusion_matrix, classification_report, roc_curve, auc,
    precision_recall_curve, average_precision_score
)

# Otimização
from sklearn.model_selection import cross_val_score, GridSearchCV

# Utilidades
import warnings
warnings.filterwarnings('ignore')

# Configurações de visualização
plt.style.use('default')
sns.set_palette('husl')
pd.set_option('display.max_columns', None)

print(" Bibliotecas importadas com sucesso!")

 Bibliotecas importadas com sucesso!


## 2. Carregamento dos Dados Pré-processados

In [2]:
print("=" * 80)
print("CARREGANDO DADOS PRÉ-PROCESSADOS")
print("=" * 80)

# Carregar conjuntos de treino e teste
X_train = pd.read_csv('../data/X_train.csv')
X_test = pd.read_csv('../data/X_test.csv')
y_train = pd.read_csv('../data/y_train.csv').values.ravel()
y_test = pd.read_csv('../data/y_test.csv').values.ravel()

print(f"\n Dados carregados!")
print(f"\nConjunto de Treino:")
print(f"  - X_train: {X_train.shape[0]} linhas x {X_train.shape[1]} features")
print(f"  - y_train: {y_train.shape[0]} exemplos")
print(f"  - Distribuição: Classe 0 = {sum(y_train == 0)} | Classe 1 = {sum(y_train == 1)}")

print(f"\nConjunto de Teste:")
print(f"  - X_test: {X_test.shape[0]} linhas x {X_test.shape[1]} features")
print(f"  - y_test: {y_test.shape[0]} exemplos")
print(f"  - Distribuição: Classe 0 = {sum(y_test == 0)} | Classe 1 = {sum(y_test == 1)}")

print(f"\nPrimeiras 5 linhas de X_train:")
X_train.head()

CARREGANDO DADOS PRÉ-PROCESSADOS

 Dados carregados!

Conjunto de Treino:
  - X_train: 80000 linhas x 41 features
  - y_train: 80000 exemplos
  - Distribuição: Classe 0 = 32002 | Classe 1 = 47998

Conjunto de Teste:
  - X_test: 20000 linhas x 41 features
  - y_test: 20000 exemplos
  - Distribuição: Classe 0 = 8000 | Classe 1 = 12000

Primeiras 5 linhas de X_train:


,age,alcohol_consumption_per_week,physical_activity_minutes_per_week,diet_score,sleep_hours_per_day,screen_time_hours_per_day,family_history_diabetes,hypertension_history,cardiovascular_history,bmi,waist_to_hip_ratio,systolic_bp,diastolic_bp,heart_rate,cholesterol_total,hdl_cholesterol,ldl_cholesterol,triglycerides,glucose_fasting,glucose_postprandial,insulin_level,hba1c,diabetes_risk_score,gender_Male,gender_Other,ethnicity_Black,ethnicity_Hispanic,ethnicity_Other,ethnicity_White,education_level_Highschool,education_level_No formal,education_level_Postgraduate,income_level_Low,income_level_Lower-Middle,income_level_Middle,income_level_Upper-Middle,employment_status_Retired,employment_status_Student,employment_status_Unemployed,smoking_status_Former,smoking_status_Never
0,-1.866152,-0.707921,0.131364,0.620577,-1.642417,-0.646764,1.886181,-0.578582,3.409731,0.107996,-0.343275,0.294063,0.459217,2.074452,-0.093029,-2.146888,0.808610,1.349645,-0.376382,0.289797,-0.361572,0.023619,0.714856,False,False,False,False,False,True,False,False,False,False,False,True,False,False,True,False,False,False
1,1.017627,-0.002589,-0.567611,0.957476,1.463692,-1.538032,-0.530172,-0.578582,-0.293278,0.358924,0.083737,1.904256,0.215439,0.402194,1.312657,0.190625,0.868508,0.196838,1.609563,-0.841596,0.585131,-0.455545,0.107890,True,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True
2,-1.289396,-0.002589,0.214294,-0.221673,-1.825130,0.609114,1.886181,-0.578582,-0.293278,-2.958902,-1.410807,-1.666172,-0.394004,0.163300,-1.186340,-0.296357,-1.048228,-0.979025,1.388902,1.324213,0.038102,1.755982,0.163068,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,True
3,-0.520388,-0.002589,-0.555764,-1.232372,-0.911568,-1.051886,1.886181,-0.578582,-0.293278,0.414685,-0.129769,0.574096,-0.637781,0.521641,0.937808,1.943760,0.239579,-0.033723,0.359153,-1.100200,0.673947,-0.738130,1.321822,True,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,True
4,0.184536,-1.413253,-0.306976,-1.400822,1.098267,-0.444203,1.886181,1.728364,-0.293278,-0.282337,-0.343275,-0.826072,1.678103,0.402194,0.219346,-1.367717,0.569018,-1.117361,0.726921,2.003049,0.795061,1.940276,1.730145,False,False,False,False,False,True,True,False,False,False,False,False,True,False,True,False,False,False


## 3. Remoção de Features Problemáticas

Removendo features que causam:
- **Data Leakage:** `diabetes_stage` (já indica o diagnóstico)
- **Multicolinearidade:** `diabetes_risk_score`, `glucose_fasting`, `glucose_postprandial`, `cholesterol_total`

In [ ]:
print("=" * 80)
print("REMOÇÃO DE FEATURES PROBLEMÁTICAS")
print("=" * 80)

print(f"\nFeatures antes da remoção: {X_train.shape[1]}")

# Features a remover
features_to_drop = [
    'diabetes_risk_score',      # Redundante (calculado a partir de outras features)
    'glucose_fasting',          # Redundante com hba1c
    'glucose_postprandial',     # Redundante com hba1c
    'cholesterol_total'         # Redundante com HDL + LDL
]

# Remover se existirem
features_removed = []
for feature in features_to_drop:
    if feature in X_train.columns:
        X_train = X_train.drop(columns=[feature])
        X_test = X_test.drop(columns=[feature])
        features_removed.append(feature)
        print(f"   Removida: {feature}")

# Remover colunas relacionadas a diabetes_stage (one-hot encoded)
diabetes_stage_cols = [col for col in X_train.columns if 'diabetes_stage' in col.lower()]
if diabetes_stage_cols:
    X_train = X_train.drop(columns=diabetes_stage_cols)
    X_test = X_test.drop(columns=diabetes_stage_cols)
    print(f"   Removidas {len(diabetes_stage_cols)} colunas de diabetes_stage (DATA LEAKAGE)")
    features_removed.extend(diabetes_stage_cols)

print(f"\nTotal de features removidas: {len(features_removed)}")
print(f"Features após remoção: {X_train.shape[1]}")

print(f"\nDataset pronto para modelagem!")

## 4. Exploração Rápida das Features Finais

In [4]:
print("=" * 80)
print("FEATURES FINAIS PARA MODELAGEM")
print("=" * 80)

print(f"\nTotal: {X_train.shape[1]} features")
print(f"\nLista de features:")
for i, col in enumerate(X_train.columns, 1):
    print(f"  {i:2d}. {col}")

FEATURES FINAIS PARA MODELAGEM

Total: 37 features

Lista de features:
   1. age
   2. alcohol_consumption_per_week
   3. physical_activity_minutes_per_week
   4. diet_score
   5. sleep_hours_per_day
   6. screen_time_hours_per_day
   7. family_history_diabetes
   8. hypertension_history
   9. cardiovascular_history
  10. bmi
  11. waist_to_hip_ratio
  12. systolic_bp
  13. diastolic_bp
  14. heart_rate
  15. hdl_cholesterol
  16. ldl_cholesterol
  17. triglycerides
  18. insulin_level
  19. hba1c
  20. gender_Male
  21. gender_Other
  22. ethnicity_Black
  23. ethnicity_Hispanic
  24. ethnicity_Other
  25. ethnicity_White
  26. education_level_Highschool
  27. education_level_No formal
  28. education_level_Postgraduate
  29. income_level_Low
  30. income_level_Lower-Middle
  31. income_level_Middle
  32. income_level_Upper-Middle
  33. employment_status_Retired
  34. employment_status_Student
  35. employment_status_Unemployed
  36. smoking_status_Former
  37. smoking_status_Never


## 5. Função Auxiliar para Avaliação de Modelos

In [ ]:
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name="Model"):
    """
    Avalia um modelo de classificação e exibe métricas completas
    """
    print("\n" + "=" * 80)
    print(f"AVALIAÇÃO: {model_name}")
    print("=" * 80)
    
    # Treinar modelo
    print("\nTreinando modelo...")
    model.fit(X_train, y_train)
    print("Treinamento concluído!")
    
    # Predições
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Probabilidades (para ROC-AUC)
    y_train_proba = model.predict_proba(X_train)[:, 1]
    y_test_proba = model.predict_proba(X_test)[:, 1]
    
    # Métricas de Treino
    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_f1 = f1_score(y_train, y_train_pred)
    train_roc_auc = roc_auc_score(y_train, y_train_proba)
    
    # Métricas de Teste
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_precision = precision_score(y_test, y_test_pred)
    test_recall = recall_score(y_test, y_test_pred)
    test_f1 = f1_score(y_test, y_test_pred)
    test_roc_auc = roc_auc_score(y_test, y_test_proba)
    
    # Exibir resultados
    print("\n" + "-" * 80)
    print("MÉTRICAS DE DESEMPENHO")
    print("-" * 80)
    
    print(f"\n{'Métrica':<20} {'Treino':<15} {'Teste':<15} {'Diferença':<15}")
    print("-" * 65)
    print(f"{'Accuracy':<20} {train_accuracy:<15.4f} {test_accuracy:<15.4f} {abs(train_accuracy - test_accuracy):<15.4f}")
    print(f"{'F1-Score':<20} {train_f1:<15.4f} {test_f1:<15.4f} {abs(train_f1 - test_f1):<15.4f}")
    print(f"{'ROC-AUC':<20} {train_roc_auc:<15.4f} {test_roc_auc:<15.4f} {abs(train_roc_auc - test_roc_auc):<15.4f}")
    
    print(f"\n{'Teste Detalhado':<20}")
    print("-" * 65)
    print(f"{'Precision':<20} {test_precision:<15.4f}")
    print(f"{'Recall':<20} {test_recall:<15.4f}")
    
    # Verificar overfitting
    if abs(train_accuracy - test_accuracy) > 0.05:
        print("\nPossível overfitting (diferença > 5% entre treino e teste)")
    else:
        print("\nModelo generaliza bem (sem overfitting significativo)")
    
    # Classification Report
    print("\n" + "-" * 80)
    print("CLASSIFICATION REPORT (Teste)")
    print("-" * 80)
    print(classification_report(y_test, y_test_pred, target_names=['Sem Diabetes', 'Com Diabetes']))
    
    # Confusion Matrix
    print("\n" + "-" * 80)
    print("CONFUSION MATRIX (Teste)")
    print("-" * 80)
    cm = confusion_matrix(y_test, y_test_pred)
    print(f"\n{'':20} {'Predito: 0':<15} {'Predito: 1':<15}")
    print(f"{'Real: 0 (Sem Diabetes)':<20} {cm[0,0]:<15} {cm[0,1]:<15}")
    print(f"{'Real: 1 (Com Diabetes)':<20} {cm[1,0]:<15} {cm[1,1]:<15}")
    
    print(f"\nInterpretação:")
    print(f"  - Verdadeiros Negativos (TN): {cm[0,0]} - Sem diabetes corretamente identificados")
    print(f"  - Falsos Positivos (FP): {cm[0,1]} - Sem diabetes identificados como COM diabetes")
    print(f"  - Falsos Negativos (FN): {cm[1,0]} - COM diabetes identificados como sem diabetes")
    print(f"  - Verdadeiros Positivos (TP): {cm[1,1]} - COM diabetes corretamente identificados")
    
    # Retornar resultados
    results = {
        'model': model,
        'model_name': model_name,
        'y_test_pred': y_test_pred,
        'y_test_proba': y_test_proba,
        'metrics': {
            'train_accuracy': train_accuracy,
            'test_accuracy': test_accuracy,
            'precision': test_precision,
            'recall': test_recall,
            'f1': test_f1,
            'roc_auc': test_roc_auc
        },
        'confusion_matrix': cm
    }
    
    return results

print("Função de avaliação criada!")

## 6. MODELO 1: Logistic Regression (Baseline)

### Por que começar com Logistic Regression?

-  **Interpretabilidade máxima** - Coeficientes indicam impacto de cada feature
-  **Baseline sólido** - Referência para comparar modelos complexos
-  **Rápido de treinar**
-  **Probabilidades calibradas** - Útil para decisões médicas
-  **Adequado para contexto médico** - Transparência é importante

In [ ]:
# Definir modelo Logistic Regression
lr_model = LogisticRegression(
    penalty='l2',              # Regularização Ridge
    C=1.0,                     # Força de regularização (1/lambda)
    class_weight='balanced',   # Ajustar para classes desbalanceadas
    max_iter=1000,             # Iterações máximas
    solver='lbfgs',            # Solver para otimização
    random_state=42,           # Reprodutibilidade
    n_jobs=-1                  # Usar todos os cores
)

print("Modelo Logistic Regression configurado!")

## 7. MODELO 2: Random Forest

### Por que Random Forest?

-  **Robusto** - Funciona bem sem muito tuning
-  **Captura não-linearidades** - Relações complexas entre features
-  **Feature importance automática** - Identifica variáveis importantes
-  **Resiste a overfitting** - Ensemble de árvores
-  **Não requer normalização** - Já fizemos, mas não seria necessário

In [ ]:
# Definir modelo Random Forest
rf_model = RandomForestClassifier(
    n_estimators=200,          # Número de árvores
    max_depth=15,              # Profundidade máxima (controla complexidade)
    min_samples_split=10,      # Mínimo de amostras para dividir nó
    min_samples_leaf=5,        # Mínimo de amostras em folha
    max_features='sqrt',       # Features a considerar em cada split
    class_weight='balanced',   # Ajustar para desbalanceamento
    random_state=42,           # Reprodutibilidade
    n_jobs=-1,                 # Usar todos os cores
    verbose=0
)

print("Modelo Random Forest configurado!")

## 8. MODELO 3: XGBoost

### Por que XGBoost?

-  **Estado da arte** - Melhor performance para dados tabulares
-  **Controle fino** - Muitos hiperparâmetros para otimizar
-  **Regularização embutida** - L1/L2 + controle de complexidade
-  **Early stopping** - Para quando não há mais melhoria
-  **Usado em produção** - Confiável e escalável

In [ ]:
# Definir modelo XGBoost
xgb_model = xgb.XGBClassifier(
    n_estimators=500,          # Número de árvores (boosting rounds)
    max_depth=6,               # Profundidade máxima
    learning_rate=0.01,        # Taxa de aprendizado (eta)
    subsample=0.8,             # Fração de amostras por árvore
    colsample_bytree=0.8,      # Fração de features por árvore
    gamma=0,                   # Regularização (mínimo loss reduction)
    reg_alpha=0,               # Regularização L1
    reg_lambda=1,              # Regularização L2
    scale_pos_weight=1,        # Ajustar se desbalanceado
    random_state=42,           # Reprodutibilidade
    eval_metric='logloss',     # Métrica para early stopping
    use_label_encoder=False,   # Evitar warning
    n_jobs=-1                  # Usar todos os cores
)

print("Modelo XGBoost configurado!")

---

## Resumo - Modelagem

### Concluído:

1. **Dados preparados:**
   - Features problemáticas removidas (data leakage, multicolinearidade)
   - Dataset pronto para treinamento

2. **Modelos configurados:**
   - Logistic Regression (Baseline)
   - Random Forest (Robusto)
   - XGBoost (Estado da arte)

3. **Função de avaliação criada:**
   - Métricas completas (Accuracy, Precision, Recall, F1, ROC-AUC)
   - Confusion Matrix
   - Classification Report

---

### Próximo Passo:

**Execute o notebook `treinamento.ipynb`** para:
- Treinar os 3 modelos
- Avaliar performance
- Analisar feature importance
- Comparar resultados
- Realizar validação cruzada
- Salvar modelos treinados